- Sử dụng SVM kernal linear để train model
- với test_size = 0.1
- Tập train 126000, tập test 1400
- Model với độ chính xác 81.92%

# Import

In [23]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

# Save model
import pickle

def _save_pkl(path, obj):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)
        
def _load_pkl(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj


# Load file

In [2]:
path = './file_pkl/v_features.pkl'
path_labels = './file_pkl/labels.pkl'
path_data = './file_pkl/dataset.pkl'
features = _load_pkl(path)
labels = _load_pkl(path_labels)
data = _load_pkl(path_data)
print(features.shape,labels.shape)

(14000, 768) (14000,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1)

In [4]:
print(len(X_train))
print(len(X_test))

12600
1400


In [67]:
data.tail(20)

,content,rate
33860,hi tikimình mua 2 hộp nhưng 1 hộp bị móp,1
2756,giao hàng nhanh gói hàng chắc_chắn,0
19752,không như quảng_cáo tệ quá,1
47147,hàng đóng_gói cẩn_thận nhưng giao hàng quá lâu...,1
12055,đóng gói sản_phẩm cẩn_thận,0
6367,hàng giao đúng ngàyđóng gói kĩ,0
51382,địa_chỉ giao hàng là hoàng ngọc hoà điện_thoại...,1
1149,sản_phẩm dùng tốt giá hợp_lý,0
12387,luc dat hang thi có hang khuyen_mai sao luc gi...,1
31986,mua 38 l mà giao 32 l không đúng như đã đặt,1


# SVM kernel linear

In [7]:
from sklearn import svm

In [8]:
model = svm.SVC(kernel='linear', probability=True, gamma=0.125)
model.fit(X_train, y_train)

sc = model.score(X_test, y_test)
print('Kết quả train model, độ chính xác = ', sc*100, '%')

Kết quả train model, độ chính xác =  81.92857142857143 %


In [10]:
_save_pkl('Model_SVM.pkl', model)

# Dự đoán bình luận


In [11]:
import torch
from transformers import AutoModel, AutoTokenizer
from underthesea import word_tokenize
import re

In [12]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Chuẩn hóa input đầu vào
- Loại bỏ dấu câu và ký tự đặc biết
- Tách từ bằng word_tokenize 
- Chuyển input về dạng vector
- Dùng phobert để trích đặc trưng
- Trả về một array các đặc trưng

In [49]:
def tokenizer_vn(row):
    return word_tokenize(row, format="text")


path_file_acr = "./TuDienVietTat/acronym_vi.txt"
acronyms_list = []
with open(path_file_acr, 'r', encoding="utf8") as f:
    for line in f.readlines():
        line = re.sub('[\n]', '', line)
        acronyms_list.append(line.split('\t'))


def replace_acronyms(self, acronyms):
    list_text = self.split(" ")
    for i in range(len(list_text)):
        for j in range(len(acronyms)):
            if(list_text[i] == acronyms[j][0]):
                list_text[i] = acronyms[j][1]
    self = " ".join(list_text)
    return self


# Hàm chuẩn hoá câu
def standardize_data(row):
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = re.sub('[\n\/]', '', row)
    # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
    row = row.replace(",", "").replace(".", "") \
        .replace(";", "").replace("“", "") \
        .replace(":", "").replace("”", "") \
        .replace('"', "").replace("'", "") \
        .replace("!", "").replace("?", "") \
        .replace("-", "").replace("?", "") \
        .replace('*\r?\n*', '')

    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    row = emoji_pattern.sub(r'', row)
    row = row.strip().lower()
    return row


def feature_text(text):
    max_len = 20
    text = standardize_data(text)
    text = replace_acronyms(text, acronyms=acronyms_list)
    sentence = tokenizer_vn(text)
    print(sentence)
    tokenized = tokenizer.encode(sentence)
    padded = np.array([tokenized + [0]*(max_len-len(tokenized))])
    train_mask = np.where(padded == 0, 0, 1)
    train_text = torch.tensor(padded).to(torch.long)
    train_mask = torch.tensor(train_mask)
    with torch.no_grad():
        features = phobert(train_text, train_mask)
    return features[0][:, 0, :].numpy()


### Func predict

In [51]:
def pre_comment(text):
    features = feature_text(text)
    sc = model.predict(features)
    if(sc[0] == 0):
        print("Tích cực")
    else:
        print("Tiêu cực")

### Input test

In [58]:
text1 = "Thời gian ship lâu, sản phẩm đóng gói kém"
pre_comment(text1)

thời_gian vận_chuyển lâu sản_phẩm đóng_gói kém
Tiêu cực


In [59]:
text2 = "Sản phẩm tốt chất lượng ship nhanh đáng để mua"
pre_comment(text2)

sản_phẩm tốt chất_lượng vận_chuyển nhanh đáng để mua
Tích cực


In [68]:
text3 = "2 tháng mới nhận đuọc hàng"
pre_comment(text3)

2 tháng mới nhận đuọc hàng
Tiêu cực
